In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
def simplex_method(A: np.array, c: np.array, b: np.array, x_0: np.array):
    # determine execution time of algorithm
    starting_time = time.time()
    
    m, n = A.shape
    
    if any(x_0 < 0): 
        x_0 = phase_1(A,b)
        print(f"New feasible point choosen: {x_0}")
    
    # calcualte calligraphic B and N (Basic- and NonBasic Index Sets)
    nonbasic_index_set = np.where(x_0==0)[0][:n-m] # Nonbasic index set (calligraphic N)
    basic_index_set = np.setdiff1d(np.arange(n), nonbasic_index_set) # Basis (calligraphic B)
    
    x = x_0
    
    df = pd.DataFrame(columns=['Iteration', 'Basic', 'Non-Basic', 'x_B','x_N', 'x_old', 'lambda', 's_N', 's_N>=0, Optimal sol.?', 'q', 'd', 'd<=0, Unbounded problem?', 'x_B_plus', 'x_N_plus', 'x_new'])
    df.loc[len(df)] = ['Init.', basic_index_set, nonbasic_index_set, '---', '---', np.round(x_0, 3), '---', '---', False, '---', '---', False, '---', '---', '---']
    
    iterations=1
    break_out_condition = -1
    """
    do simplex method until either:
        - find optimal point (0)
        - detect unboundness (1)
    """
    while True:
        # calcualte B, N
        B = A[:, basic_index_set]
        # check if B is invertible
        if np.linalg.det(B) == 0: raise ValueError(f'B is not invertible')
        N = A[:, nonbasic_index_set]

        # calcualte x_B, x_N
        x_B = np.linalg.inv(B) @ b
        # check x_B>=0 constraint
        if not all(x >= 0 for x in x_B): raise ValueError(f'x_B>=0 constraint is violated for x={x}')
        
        x_N = np.zeros(c.shape[0] - b.shape[0])

        # calcualte c_B and c_N
        c_B = c[basic_index_set]
        c_N = c[nonbasic_index_set]

        # get lambda
        lambda_ = np.linalg.solve(B.T, c_B)

        # calculate s_N
        s_N = np.subtract(c_N, (N.T @ lambda_))
        
        # 1st exit condition: Optimal point found
        if np.all(s_N >= 0): 
            break_out_condition = 0
            df.loc[len(df)] = [iterations, basic_index_set, nonbasic_index_set, np.round(x_B, 3), np.round(x_N, 3), np.round(x, 3), np.round(lambda_, 3), np.round(s_N, 3), True, 
                               '---', '---', False, '---', '---', '---']
            return df, x, f'Algorithm took {(time.time() - starting_time) * 1000} ms to run'

        # select q: min index from s_N
        q_min_idx = nonbasic_index_set[np.argmin(s_N)] 
        
        # get d
        A_q = A[:, q_min_idx] 
        d = np.linalg.solve(B, A_q)

        # 2nd exit condition: problem is unbounded
        if np.all(d <= 0): 
            break_out_condition = 1
            df.loc[len(df)] = [iterations, basic_index_set, nonbasic_index_set, np.round(x_B, 3), np.round(x_N, 3), np.round(x, 3), np.round(lambda_, 3), np.round(s_N, 3), False, 
                               q_min_idx, d, False, np.round(x_B_plus, 3), np.round(x_N_plus, 3), np.round(x_new, 3)]
            return df, _, f'Algorithm took {(time.time() - starting_time) * 1000} ms to run'

        # calcualte x_q^+ and p=min index from all possible ones
        d_pos_idx = [idx for idx in range(len(d)) if d[idx] > 0]
        
        # check error condition for q
        if len(d_pos_idx) == 0: raise ValueError('Could not find any appropriate d')
        d_min_idx = d_pos_idx
        
        # get from all x_q_plus the one with minimal val
        x_q_plus = np.min(x_B[d_min_idx] / d[d_min_idx]) 
        p_min_idx = d_min_idx[np.argmin(x_B[d_min_idx] / d[d_min_idx])] 

        # update x_B^+
        x_B_plus = np.subtract(x_B, (d * x_q_plus))

        # update x_N_^+
        x_N_plus = x_N
        entering_var = -1 # needed for updating N and x_N_plus

        entering_var = np.where(nonbasic_index_set == q_min_idx)[0]
        # check for validity of varaible that changes N and x_N_plus
        if entering_var == -1: raise ValueError('Entering variable for N is undefined')
        x_N_plus[entering_var] = x_q_plus
        
        # update x according to indices
        x_new = x.copy()
        x_new[basic_index_set] = x_B_plus
        x_new[nonbasic_index_set] = x_N_plus

        # update basic- and nonbasic index set
        basic_index_set_copy = basic_index_set.copy()
        basic_index_set[p_min_idx] = q_min_idx
        nonbasic_index_set[entering_var] = basic_index_set_copy[p_min_idx]       
        
        df.loc[len(df)] = [iterations, basic_index_set, nonbasic_index_set, np.round(x_B, 3), np.round(x_N, 3), np.round(x, 3), np.round(lambda_, 3), np.round(s_N, 3), False, 
                            q_min_idx, d, False, np.round(x_B_plus, 3), np.round(x_N_plus, 3), np.round(x_new, 3)]
        x = x_new
        iterations += 1

In [4]:
A_con = np.array([[1,1],
              [2,1/2]])

m = A_con.shape[0]
A = np.hstack((A_con, np.eye(m)))
b = np.array([5, 8], dtype=np.float64)
c = np.array([-4, -2], dtype=np.float64)

# extends c to 4d (same dim as A)
missing_dims_c = A.shape[1] - c.shape[0]
c = np.append(c, np.array([0] * missing_dims_c))

x_0 = np.array([0, 0], dtype=np.float64)
x_star = b- np.dot(A_con, x_0)
x_0= np.hstack((x_0,x_star))


df, optimal_point, execution_time = simplex_method(A=A, c=c, b=b, x_0=x_0)
df

,Iteration,Basic,Non-Basic,x_B,x_N,x_old,lambda,s_N,"s_N>=0, Optimal sol.?",q,d,"d<=0, Unbounded problem?",x_B_plus,x_N_plus,x_new
0,Init.,"[1, 0]","[3, 2]",---,---,"[0.0, 0.0, 5.0, 8.0]",---,---,False,---,---,False,---,---,---
1,1,"[1, 0]","[3, 2]","[5.0, 8.0]","[4.0, 0.0]","[0.0, 0.0, 5.0, 8.0]","[0.0, 0.0]","[-4.0, -2.0]",False,0,"[1.0, 2.0]",False,"[1.0, 0.0]","[4.0, 0.0]","[4.0, 0.0, 1.0, 0.0]"
2,2,"[1, 0]","[3, 2]","[1.0, 4.0]","[0.0, 1.333]","[4.0, 0.0, 1.0, 0.0]","[0.0, -2.0]","[2.0, -1.0]",False,1,"[0.75, 0.25]",False,"[0.0, 3.667]","[0.0, 1.333]","[3.667, 1.333, 0.0, 0.0]"
3,3,"[1, 0]","[3, 2]","[1.333, 3.667]","[0.0, 0.0]","[3.667, 1.333, 0.0, 0.0]","[-1.333, -1.333]","[1.333, 1.333]",True,---,---,False,---,---,---


In [5]:
execution_time

'Algorithm took 36.98325157165527 ms to run'

In [6]:
A_con = np.array([[1,1],
              [2,1/2]])

m = A_con.shape[0]
A = np.hstack((A_con, np.eye(m)))
b = np.array([5, 8], dtype=np.float64)
c = np.array([-5, -1], dtype=np.float64)

# extends c to 4d (same dim as A)
missing_dims_c = A.shape[1] - c.shape[0]
c = np.append(c, np.array([0] * missing_dims_c))

x_0 = np.array([0, 0], dtype=np.float64)
x_star = b- np.dot(A_con, x_0)
x_0= np.hstack((x_0,x_star))


df, optimal_point, execution_time = simplex_method(A=A, c=c, b=b, x_0=x_0)
df

,Iteration,Basic,Non-Basic,x_B,x_N,x_old,lambda,s_N,"s_N>=0, Optimal sol.?",q,d,"d<=0, Unbounded problem?",x_B_plus,x_N_plus,x_new
0,Init.,"[2, 0]","[3, 1]",---,---,"[0.0, 0.0, 5.0, 8.0]",---,---,False,---,---,False,---,---,---
1,1,"[2, 0]","[3, 1]","[5.0, 8.0]","[4.0, 0.0]","[0.0, 0.0, 5.0, 8.0]","[0.0, 0.0]","[-5.0, -1.0]",False,0,"[1.0, 2.0]",False,"[1.0, 0.0]","[4.0, 0.0]","[4.0, 0.0, 1.0, 0.0]"
2,2,"[2, 0]","[3, 1]","[1.0, 4.0]","[0.0, 0.0]","[4.0, 0.0, 1.0, 0.0]","[0.0, -2.5]","[2.5, 0.25]",True,---,---,False,---,---,---


In [8]:
execution_time

'Algorithm took 23.992538452148438 ms to run'